In [1]:
import tensorflow as tf
import random

In [2]:
( x_train, y_train ), ( x_test, y_test ) = tf.keras.datasets.mnist.load_data()

x_train = tf.cast( x_train / 255.0, tf.float32 )
x_test = tf.cast( x_test / 255.0, tf.float32 )

y_train = tf.keras.utils.to_categorical( y_train, 10 )
y_test = tf.keras.utils.to_categorical( y_test, 10 )
print( x_train.shape, x_test.shape )

(60000, 28, 28) (10000, 28, 28)


In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
W1 = tf.Variable( tf.random.normal( [ 3, 3, 1, 32 ], stddev = 0.01 ) )
def Layer1( X, rate ):
    l1 = tf.nn.conv2d( X, W1, strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
    l1 = tf.nn.relu( l1 )
    l1 = tf.nn.max_pool( l1, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' )
    return ( tf.nn.dropout( l1, rate ) )

W2 = tf.Variable( tf.random.normal( [ 3, 3, 32, 64 ], stddev = 0.01 ) )
def Layer2( X, rate ):
    l2 = tf.nn.conv2d( Layer1( X, rate ), W2, strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
    l2 = tf.nn.relu( l2 )
    l2 = ( tf.nn.max_pool( l2, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' ) )
    return ( tf.nn.dropout( l2, rate ) )

W3 = tf.Variable( tf.random.normal( [ 3, 3, 64, 128 ], stddev = 0.01 ) )
def Layer3( X, rate ):
    l3 = tf.nn.conv2d( Layer2( X, rate ), W3, strides = [ 1, 1, 1, 1 ], padding = 'SAME' )
    l3 = tf.nn.relu( l3 )
    l3 = ( tf.nn.max_pool( l3, ksize = [ 1, 2, 2, 1 ],
                           strides = [ 1, 2, 2, 1 ], padding = 'SAME' ) )
    return ( tf.nn.dropout( l3, rate ) )

def Layer3_flat( X, rate ):
    return ( tf.reshape( Layer3( X, rate ), [ -1, 128 * 4 * 4 ] ) )

xavier = tf.keras.initializers.GlorotUniform()

W4 = tf.Variable( xavier( [ 128 * 4 * 4, 625 ] ) )
b4 = tf.Variable( tf.random.normal( [ 625 ] ) )
def Layer4( X, rate ):
    l4 = tf.nn.relu( tf.matmul( Layer3_flat( X, rate ), W4 ) + b4 )
    return  ( tf.nn.dropout( l4, rate ) )

W5 = tf.Variable( xavier( [ 625, 10 ] ) )
b5 = tf.Variable( tf.random.normal( [ 10 ] ) )
def Logits( X, rate ):
    return ( tf.matmul( Layer4( X, rate ), W5 ) + b5 )

In [5]:
@tf.function
def Cost( X, Y, rate ):
    return ( tf.reduce_mean( 
        tf.nn.softmax_cross_entropy_with_logits(
            logits = Logits( X, rate ), labels = Y )
    ) )

def Minimize( X, Y, rate ):
    loss = lambda: Cost( X, Y, rate )
    
    tf.keras.optimizers.Adam( learning_rate ).minimize( loss, 
                                                      [ W1, W2, W3,
                                                       b4, b5 ] )

In [6]:
print( 'Learning started. It takes sometime' )
for epoch in range( training_epochs ):
    avg_cost = 0
    total_batch = int( len( x_train ) / batch_size )
    
    start_batch, end_batch = 0, batch_size
    for i in range( total_batch ):
        batch_xs, batch_ys = \
            x_train[ start_batch : end_batch ], y_train[start_batch : end_batch ]
        batch_xs = tf.reshape( batch_xs, [ -1, 28, 28, 1 ] )
        
        Minimize( batch_xs, batch_ys, 0.3 )
        cost_val = Cost( batch_xs, batch_ys, 0.3 )
        
        avg_cost += cost_val / total_batch
        
        start_batch = start_batch + batch_size
        end_batch = end_batch + batch_size
    
    print( 'Epoch: {:04d}, Cost: {:.9f}'.format( epoch + 1, avg_cost ) )
print( 'Learning Finished!' )

Learning started. It takes sometime
Epoch: 0001, Cost: 0.776947200
Epoch: 0002, Cost: 0.273817807
Epoch: 0003, Cost: 0.220963582
Epoch: 0004, Cost: 0.192832544
Epoch: 0005, Cost: 0.174315453
Epoch: 0006, Cost: 0.159904182
Epoch: 0007, Cost: 0.148792610
Epoch: 0008, Cost: 0.137430981
Epoch: 0009, Cost: 0.128233299
Epoch: 0010, Cost: 0.121030472
Epoch: 0011, Cost: 0.118418634
Epoch: 0012, Cost: 0.110297635
Epoch: 0013, Cost: 0.110754788
Epoch: 0014, Cost: 0.103921674
Epoch: 0015, Cost: 0.103045061
Learning Finished!


In [7]:
def CorrectPrediction( X, Y ):
    return ( tf.equal( tf.argmax( Logits( X, 0 ), 1 ), tf.argmax( Y, 1 ) ) )

def Accuracy( X, Y ):
    return ( tf.reduce_mean( tf.cast( CorrectPrediction( X, Y ), tf.float32 ) ) )

def Evaluate( X_sample, Y_sample, batch_size = 512 ):
    N = X_sample.shape[0]
    correct_sample = 0
    
    for i in range( 0, N, batch_size ):
        X_batch = X_sample[ i : i + batch_size ]
        Y_batch = Y_sample[ i : i + batch_size ]
        N_batch = X_batch.shape[ 0 ]
        
        correct_sample += Accuracy( X_batch, Y_batch ) * N_batch
        
    return correct_sample / N

In [8]:
x_img = tf.reshape( x_test, [ -1, 28, 28, 1 ] )
tf.print( 'Accuracy: ', Accuracy( x_img, y_test ) )
tf.print( 'Accuracy: ', Evaluate( x_img, y_test ) )

Accuracy:  0.9866
Accuracy:  0.9866


In [9]:
r = random.randint( 0, len( x_test ) - 1 )
x_img = tf.reshape( x_test[ r: r + 1], [ -1, 28, 28, 1] )
tf.print( 'Label: ', tf.argmax( y_test[ r: r + 1 ], 1 ) )
tf.print( 'Prediction: ',
     tf.argmax( Logits( x_img, 0 ), 1 ) )

Label:  [0]
Prediction:  [0]
